# Cloud Data Transfer Speeds Benchmarking Workflow

Add overview of workflow

## Step 0: Load Required Setup Packages & Classes

Enter the following parameters to install packages to the correct conda instance and environment.


`jupyter_conda_path : str`
    - The path to miniconda that this Jupyter notebook is running from. Do not include a terminal `/` at the end of the path.
    
`jupyter_conda_env : str`
    - The conda environment that this Jupyter notebook is running in

In [1]:
jupyter_conda_path = "/home/jgreen/.miniconda3c"
jupyter_conda_env = "jupyter"

Installs required workflow setup packages and calls UI generation script. If one or more of the packages don't exist in the specified environment, they will install for you. Note that if installation is required, this cell will take a few minutes to complete execution.

**NOTE: If you recieve an import error for `jupyter-ui-poll`, you will have to manually install the package in a user container terminal with the following commands:**
```
source <jupyter_conda_path>/etc/profile.d/conda.sh
conda activate <jupyter_conda_env>
pip install jupyter-ui-poll
```

In [2]:
import os
import json
import sys

print('Checking conda environment for UI depedencies...')
os.system("bash " + os.getcwd() + f"/jupyter-helpers/install_ui_packages.sh {jupyter_conda_path} {jupyter_conda_env}")
print('All dependencies installed.')

sys.path.insert(0, os.getcwd() + '/jupyter-helpers')
import ui_helpers as ui
import pandas as pd

Checking conda environment for UI depedencies...
All dependencies installed.


## Step 1: Define Workflow Inputs

Run the following cells to generate interactive widgets allowing you to enter all workflow inputs. **All inputs must be filled out to proceed with the benchmarking process.**

### Cloud Resources

#### Compute Resources

Before defining anything else, the resources you intend to use with the benchmarking must be defined. Currently, only resources defined in the Parallel Works platform may be used. Also of note are options that will be passed to Dask: you must specify the number of cores and memory per worker node in the cluster. Without these values, Dask will not be able to submit jobs.

In particular, these options are included so that you can form fair comparisons between different cloud service providers (CSPs). Generally, different CSPs won't have worker nodes with the exact same specs, and in order to achieve a fair comparison between two CSPs one cluster must be limited such that it does not exceed the computational power of the other.

Before submitting your options, read and adhere to the following guidelines:

- **When entering the number of CPUs in the following widget, you must know if the worker nodes of your clusters have hyperthreading enabled.** The instance type of a partition in the resource definition will display a number of vCPUs: this number represents the number of CPUs with hyperthreading enabled. **If hyperthreading is disabled, enter the number of vCPUs divided by 2 into this workflow.** If you do not know, assume that hyperthreading is disabled.

- Supply a path to an existing miniconda installation or the desired installation path. Leave the default option of `~` to install to the home directory of the cluster. **Miniconda must be installed in a directory accessible to the head node and all worker nodes. If you are unsure if your current/desired installation directory is a shared directory, use the default option.**

<div class="alert alert-block alert-info">
For resources controlled from the Parallel Works platform, the <code>Resource name</code> box should be populated with the name found on the <b>RESOURCES</b>  tab. Additional information such as memory and cores can be found from the respective resource definition.
    </div>

In [3]:
resource = ui.resourceWidgets()
resource.display()
resources = resource.processInput()
print(f'Your resource inputs:\n {resources}')

Accordion(children=(VBox(children=(HBox(children=(Label(value='Resource Name: '), Text(value=''))), HBox(child…

Button(description='Submit', style=ButtonStyle())


-----------------------------------------------------------------------------
If you wish to change information about cloud resources, run this cell again.

Your resource inputs:
 [{'Name': 'gcptestnew', 'MinicondaDir': '~', 'CSP': 'GCP', 'Dask': {'Scheduler': 'SLURM', 'Partition': 'compute', 'CPUs': 2, 'Memory': 16.0}}]


#### Object Stores

This set of inputs is where you enter the cloud object store Universal Resource Identifiers (URIs). Both public and private buckets are supported. For the latter, ensure that you have access credentials with *at least* read, write, list, and put (copy from local storage to cloud) permissions, as format conversions will need to be made during the benchmarking process.

Be sure to double-check your inputs to ensure that the bucket names and credentials are correct!

In [4]:
store = ui.storageWidgets()
store.display()
storage = store.processInput()

# Following line should be commented out if you don't want AWS S3 credentials shown in plain text
print(f'Your storage inputs:\n {storage}')

Accordion(children=(VBox(children=(HBox(children=(Label(value='Storage URI: '), Text(value='', placeholder='gc…

Button(description='Submit', style=ButtonStyle())


--------------------------------------------------------------------------------------
If you wish to change information about cloud storage locations, run this cell again.

Your storage inputs:
 [{'Path': 'gs://cloud-data-benchmarks', 'Type': 'Private', 'CSP': 'GCP', 'Credentials': {'token': './.cloud-data-benchmarks.json'}}]


### Datasets

#### User-Supplied Datasets

Below you can specify datasets that you want to be tested in the benchmarking. You can either enter single files or multiple files that belong to a single dataset, provided that the dataset matches one of the supported formats. **Read the following input rules after running the UI cell below this one.**


1. Activate the checkbox if you desire to record your user-defined datasets. **If it is not checked, none of your inputs will be recorded.**

<br>

2. If speciying data in a bucket, input the path to the data within the bucket, *not* the full URI (i.e., use `path/to/file.extension` and not `<URI prefix>://<bucketname>/path/to/file.extension`)

<br>

3. Use absolute paths for data stored in the user container (or a filesystem mounted in the user container): `/path/to/data.extension`

<br>

4. Use globstrings (`path/to/files/*`) to specify datasets that are split up into multiple subfiles.
    - If using a globstring, ensure that *only* files that belong to the dataset exist in that directory. The workflow will take all files in the directory before the `*` and attempt to gather them into a single dataset.
    - **Globstrings are NOT supported for NetCDF files**
    
<br>

5. If you have a dataset stored in multiple cloud storage locations that will be used in the benchmarking, you must define an input for each of the locations. That is, you must define each location of the data separately.

In [5]:
userdata = ui.userdataWidgets(storage=storage)
userdata.display()
user_files = userdata.processInput()
print(f'Your data inputs:\n {user_files}')

Checkbox(value=False, description='Provide datasets to workflow?')

Accordion(children=(VBox(children=(HBox(children=(Label(value='Data Format'), Dropdown(options=('NetCDF4', 'CS…

Button(description='Submit', style=ButtonStyle())


---------------------------------------------------------------------------
If you wish to change information about your input data, run this cell again.

Your data inputs:
 []


#### Randomly-Generated Datasets

Another option to supply data to the benchmarking is to create randomly-generated datasets. CSV datasets can be as large as you want and provide a great option if you are new to the world of cloud-native data formats. There are currently two supported randomly-generated data formats: CSV and NetCDF4. Since NetCDF4 is a gridded data format, options to specify the number of coordinate axes and data variables are also included.

<div class="alert alert-block alert-info">
Randomly-generated NetCDF4 file sizes are limited by available disk space in the cluster you are generating the file with. Ensure that you have adequate disk space in your cluster, or the file will not fully generate.
    </div>

In [6]:
randgen = ui.randgenWidgets(resources=resources)
randgen.display()
randfiles = randgen.processInput()
print(f'Your randomly-generated file options:\n {randfiles}')

Accordion(children=(HBox(children=(Label(value='Resource to write random files with: '), Dropdown(options=('gc…

Button(description='Submit', style=ButtonStyle())


-------------------------------------------------------------------------------
If you wish to change the randomly generated file options, run this cell again.

Your randomly-generated file options:
 [{'Format': 'CSV', 'Generate': False, 'SizeGB': 0.0}, {'Format': 'NetCDF4', 'Generate': False, 'SizeGB': 0.0, 'Data Variables': 1.0, 'Float Coords': 2.0}, {'Resource': 'gcptestnew'}]


### Benchmark Options

#### Legacy to Cloud-Native Conversion Options

There are a number of different options to choose from when legacy files (e.g., CSV, NetCDF) are converted to cloud-native (e.g., Parquet, Zarr). In fact, this input field is where you have the most control over the type of resutls you want to see. You may specify different compression algorithms, compression levels, and chunksizes to use when cloud-native files are written to cloud storage, which each have different effects on how fast files will be written and read.

Note that, for large chunksizes, the workflow may fail. This is an intended consequence, as the workflow is designed to let you explore the effects of different options on your data. **In general, it is recommended that you keep chunksizes in the range of 50-150 megabytes**. Anything higher will result in either poor performance or out-of-memory errors. Similarly, some compression algorithms will result in poor performance.

The widget below allows you to define different sets of options to convert files with. To make multiple selections in the compression algorithm and dataset fields, simply hold `SHIFT` and/or `CTRL` (or `COMMAND` on Mac). Using these option sets, the files defined in each set will be converted using the specified compression & chunksize information. The following rules apply:

- An individual dataset will be converted `(# of compression algorithms in option set 1)*...*(# of compression algorithms in option set n)` times
- The total number of conversions is equal to the sum of individual dataset conversions
- Datasets will only be converted if they are selected in at least one option set.
- **If you want to keep the internal chunking scheme of a NetCDF4 dataset, set chunksize to 0**

Gridded data chunk dimensions are computed as n-dimensional cubes (i.e., a data variable described by 3 dimensions will have a 3-dimensional chunk of uniform dimension lengths, and so on for higher-order cases). This may give suboptimal chunk sizes across the entire dataset (i.e., there may be more "filler" chunks: chunks that are used to fill the spaces where desired chunk dimensions cannot fit)

In [7]:
convertOptions = ui.convertOptions(user_files, randfiles)
convertOptions.display()
convert_options = convertOptions.processInput()
print(f'Your cloud-native data options:\n {convert_options}')

Accordion(children=(VBox(children=(HBox(children=(Label(value='Data Format Type: '), Dropdown(options=('Gridde…

Button(description='Submit', style=ButtonStyle())


---------------------------------------------------------------------------
If you wish to change information about your conversion options, run this cell again.

Your cloud-native data options:
 [{'Algorithms': (), 'Level': 5, 'Chunksize': 120.0, 'Datasets': ()}]


#### Core Options

Finally, there are a couple important options that apply to the core of the benchmarks. These are:

- `worker_step : int` - When performing file reads (option not yet implemented in conversions), the workflow will loop through a range of Dask workers (or number of parallel reads). The loop starts at the maximum defined worker amount, and reduces by `worker_step` until the lowest possible value of workers is reached.

</br>

- `tests : int` - The number of times each individual file will be read for measurement. Entering a number greater than 1 will take much longer to run, but results will reflect the volaility in network performance you can expect when using the data for computations.

In [8]:
worker_step = 5
tests = 1

## Step 2: Notebook Setup

Executing the following cell will write all of your inputs to `inputs.json`, install miniconda3 and the "cloud-data" Python environment to all resources, and write randomly-generated files to all cloud storage locations (if any files were specified). If writing randomly-generated files, especially large ones, the execution of this cell may take a while.

<div class="alert alert-block alert-info">
While randomly-generated files are written in parallel by default, if you wish to speed up the execution of this cell, consider creating/choosing a resource with more powerful worker nodes.
    </div>

In [10]:
print('Setting up workflow...')

user_input = json.dumps({"RESOURCES" : resources,
                         "STORAGE" : storage,
                         "USERFILES" : user_files,
                         "RANDFILES" : randfiles,
                         "CONVERTOPTS" : convert_options,
                         "GLOBALOPTS" : {'worker_step' : worker_step,
                                         'tests' : tests,
                                        'local_conda_sh' : jupyter_conda_path}
                        })

with open('inputs.json', 'w') as outfile:
    outfile.write(user_input)

os.system("bash workflow_notebook_setup.sh")

print('Workflow setup complete.')

Setting up workflow...
Will install miniconda3 to "/home/jgreen/.miniconda3"
Installing Miniconda-23.5.2 on "gcptestnew"...
--2023-08-11 19:41:13--  https://repo.anaconda.com/miniconda/Miniconda3-py311_23.5.2-0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103219356 (98M) [application/x-sh]
Saving to: ‘/tmp/miniconda-1691782873-1637.sh’

     0K .......... .......... .......... .......... ..........  0% 3.76M 26s
    50K .......... .......... .......... .......... ..........  0% 10.1M 18s
   100K .......... .......... .......... .......... ..........  0% 5.42M 18s
   150K .......... .......... .......... .......... ..........  0% 11.1M 16s
   200K .......... .......... .......... .......... ..........  0% 13.0M 14s
   250K .......... .......... .......... .......... .........

 24050K .......... .......... .......... .......... .......... 23%  476M 0s
 24100K .......... .......... .......... .......... .......... 23%  479M 0s
 24150K .......... .......... .......... .......... .......... 24%  192M 0s
 24200K .......... .......... .......... .......... .......... 24%  430M 0s
 24250K .......... .......... .......... .......... .......... 24%  474M 0s
 24300K .......... .......... .......... .......... .......... 24%  415M 0s
 24350K .......... .......... .......... .......... .......... 24%  372M 0s
 24400K .......... .......... .......... .......... .......... 24%  452M 0s
 24450K .......... .......... .......... .......... .......... 24%  490M 0s
 24500K .......... .......... .......... .......... .......... 24%  505M 0s
 24550K .......... .......... .......... .......... .......... 24%  451M 0s
 24600K .......... .......... .......... .......... .......... 24%  489M 0s
 24650K .......... .......... .......... .......... .......... 24%  455M 0s
 24700K ....

Finished installing Miniconda on "gcptestnew".                                                                   

Building "cloud-data" environment on "gcptestnew"...
Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
  current version: 23.5.2
  latest version: 23.7.2

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.2





bzip2-1.0.8          | 105 KB    |            |   0% 
snappy-1.1.9         | 870 KB    |            |   0% 

freetype-2.10.4      | 890 KB    |            |   0% 


yarl-1.8.1           | 94 KB     |            |   0% 



qt-main-5.15.2       | 59.6 MB   |            |   0% 




cloudpickle-2.2.1    | 27 KB     |            |   0% 





brotli-bin-1.0.9     | 19 KB     |            |   0% 






libbrotlidec-1.0.9   | 33 KB     |            |   0% 







matplotlib-inline-0. | 18 KB     |            |   0% 








cryptography-41.0.2  | 2.0 MB    |            |   0% 









prompt-toolkit-3.0.3 | 707 KB    |            |   0% 










abseil-cpp-20211102. | 1.1 MB    |            |   0% 











utf8proc-2.6.1       | 302 KB    |            |   0% 












pure_eval-0.2.2      | 14 KB     |            |   0% 













libpng-1.6.39        | 304 KB    |            |   0% 














cftime-1.6.2         | 233 KB    |            |   0% 















google-cloud-co

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Installing pip dependencies: ...working... Ran pip subprocess with arguments:
['/home/jgreen/.miniconda3/envs/cloud-data/bin/python', '-m', 'pip', 'install', '-U', '-r', '/home/jgreen/condaenv._mkkg9z8.requirements.txt', '--exists-action=b']
Pip subprocess output:
  Using cached msgpack-1.0.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (325 kB)
  Obtaining dependency information for scipy==1.11.1 from https://files.pythonhosted.org/packages/b8/46/1d255bb55e63de02f7b2f3a2f71b59b840db21d61ff7cd41edbfc2da448a/scipy-1.11.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached scipy-1.11.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (59 kB)
Using cached scipy-1.11.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.2 MB)
  Attempting uninstall: msgpack
    Found existing installation: ms

## Step 3: Run Benchmarking

### Convert File to Cloud-Native

Since one of the major goals of this benchmarking is testing legacy formats against cloud-native ones, we must convert your legacy-formatted data (CSV and NetCDF4) into their corresponding cloud-native formats. This cell will execute and time the conversion process, writing each new format in parallel. The conversion will be done using each cluster's full amount of resources, so be mindful of this feature when using clusters that are expensive to operate.

In [11]:
os.system("bash benchmarks-core/run_benchmark_step.sh \"convert-data.py\" \"conversions.csv\"")
df_conversions = pd.read_csv(os.getcwd() + '/results/csv-files/conversions.csv')
df_conversions

/home/jgreen/.miniconda3/envs/cloud-data/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39692 instead
  warnings.warn(


Waiting for worker nodes to start up...


cat: /home/jgreen/pw/storage/cloud-data-transfer-benchmarking/results/csv-files/results_tmp.csv: No such file or directory
rm: cannot remove ‘/home/jgreen/pw/storage/cloud-data-transfer-benchmarking/results/csv-files/results_tmp.csv’: No such file or directory


,resource,resource_csp,bucket,bucket_csp,conversionType,orig_dataset_name,data_vars,compr_alg,compr_lvl,chunksize_MB,conversion_time
0,gcptestnew,GCP,gs://cloud-data-benchmarks,GCP,CSV-to-Parquet,random_5.0GB_CSV,NaN,snappy,5,120.0,57.436903


### File Reads

The last computation-intensive test in the benchmarking is reading and timing files from cloud storage. This will give you an idea of what data transfer throughput you can expect when using cloud storage and different data formats in other workflows.

In [ ]:
os.system("bash benchmarks-core/run_benchmark_step.sh \"read-data.py\" \"reads.csv\"")
df_reads = pd.read_csv(os.getcwd() + '/results/csv-files/reads.csv')
df_reads

## TODO: Step 4: Visualize Results

**Feature not ready**

## Step 5: Remove Benchmarking Files from Cloud Resources (Optional)

Running the following cell will remove all files that the benchmark has copied/written to both clusters and cloud storage. It will not delete any of the original user-supplied datasets.

In [ ]:
os.system("bash postprocessing/remove-benchmark-files.sh")